В этом ноубуке используются обeченные тестовые модели, с их помощью происходит сэмплирование и далее закодированные категориальные переменные переводятся в изначальные. После чего замеряется качество.

Способ нормализации: quantile transformation для числовых признаков и standart scale для категориальных OHE

Добавление шума: ко всем признакам

In [1]:
%cd ..

/workspace/TabDDPM_copy/TabDDPM


/venv/main/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import json
import pandas as pd
import torch
import matplotlib.pyplot as plt

from utils import *
from models.tabddpm_ON_QnSC.tabddpm_ON_QnSC import *

from tqdm.notebook import tqdm

from eval.base_metrics import calculate_base_metrics
from eval.similarity import calculate_similarity
from eval.mle import calculate_mle
from eval.alpha_beta import calculate_alpha_beta
from eval.detection import calculate_detection
from eval.dcr import calculate_DCR

### Подготовка

In [3]:
dataname = 'adult_ON_QnSC_200k'
model_short = 'tabddpm_ON_QnSC'
model_name = 'tabddpm_ON_QnSC'
model_name_postfix = '_200k'
dataname_initial = 'adult'

In [4]:
if torch.cuda.is_available():
    device = f'cuda'
else:
    device = 'cpu'
CONFIG.add_arg('device', device)
print(f"Device: {device}")

CONFIG.add_arg('dataname', dataname)
CONFIG.add_arg('real_path',
                   f'./synthetic/{dataname}/initial_real.csv')
CONFIG.add_arg('test_path',
                   f'./synthetic/{dataname}/initial_test.csv')

CONFIG.add_arg('info_path',
                   f'./data/{dataname}/initial_info.json')

Device: cuda


In [5]:
model_save_path_hist = []
for sigma in [0.01, 0.25, 0]:
    model_save_path=f"./models/{model_name}/ckpt/{dataname_initial}/{model_name}{model_name_postfix}_const_{str(sigma).replace('.', '_')}"
    model_save_path_hist.append(model_save_path)

for i in [5,6, 10]:
    model_save_path=f"./models/{model_name}/ckpt/{dataname_initial}/{model_name}{model_name_postfix}_mult_{i}"
    model_save_path_hist.append(model_save_path)

In [6]:
from pickle import dump, load
with open(f'data/{dataname}/normalizers.json', 'r') as f:
        normalizers = json.load(f)
normalizers['num_normalizer'] = load(open(f'./data/{dataname}/num_normalizer_{dataname}.pkl', 'rb'))
normalizers['cat_normalizer'] = load(open(f'./data/{dataname}/cat_normalizer_{dataname}.pkl', 'rb'))
print(f"normalizers received from `data/{dataname}` folder")

normalizers received from `data/adult_ON_QnSC_200k` folder


In [7]:
normalizers

{'len_num_prev': 6,
 'len_cat_prev': 102,
 'len_target_prev': 1,
 'num_normalizer': QuantileTransformer(output_distribution='normal', random_state=0,
                     subsample=1000000000),
 'cat_normalizer': StandardScaler()}

### Sample + Eval

In [8]:
CONFIG.get_all_args()

{'dataname': 'adult_ON_QnSC_200k',
 'method': None,
 'device': 'cuda',
 'mode': 'train',
 'train': 1,
 'sample_save_path': None,
 'sigma_scheduller_name': 'constant',
 'sigma_value': 0.001,
 'num_noise': 103,
 'real_path': './synthetic/adult_ON_QnSC_200k/initial_real.csv',
 'test_path': './synthetic/adult_ON_QnSC_200k/initial_test.csv',
 'info_path': './data/adult_ON_QnSC_200k/initial_info.json',
 'save_path': None}

In [9]:
model_save_path

'./models/tabddpm_ON_QnSC/ckpt/adult/tabddpm_ON_QnSC_200k_mult_10'

In [10]:
overall_metrics = {}

for model_save_path in model_save_path_hist:
    model_name = model_save_path.split('/')[-1]
    sample_save_path = f'./synthetic/{dataname}/{model_name}.csv'

    CONFIG.add_arg('method', model_name)
    CONFIG.add_arg('sample_save_path',
                       f"synthetic/{CONFIG.get_arg('dataname')}/{CONFIG.get_arg('method')}.csv")

    overall_metrics[model_name] = {}

    print(model_save_path, model_name, sample_save_path)

    tabddpm_noise_ohe = TabDDPM_OHE_Noise_QnSC(CONFIG, model_save_path=model_save_path, sigmas=None)
    tabddpm_noise_ohe.sample(sample_save_path=sample_save_path)
    postsample_OHE(dataname, f'./synthetic/{dataname}/initial_{model_name}.csv', normalizers=normalizers)

    # подсчет метрик
    try:
        overall_metrics[model_name]['base_metrics'] = calculate_base_metrics(make_binary=True, value=' >50K') # ошибка
    except Exception as e:
        overall_metrics[model_name]['base_metrics'] = np.nan
    overall_metrics[model_name]['similarity'] = calculate_similarity()
    overall_metrics[model_name]['mle'] = calculate_mle()
    overall_metrics[model_name]['detection'] = calculate_detection()
    overall_metrics[model_name]['DCR'] = calculate_DCR()
    overall_metrics[model_name]['quality'] = calculate_alpha_beta()

./models/tabddpm_ON_QnSC/ckpt/adult/tabddpm_ON_QnSC_200k_const_0_01 tabddpm_ON_QnSC_200k_const_0_01 ./synthetic/adult_ON_QnSC_200k/tabddpm_ON_QnSC_200k_const_0_01.csv
No NaNs in numerical features, skipping
mlp
Sample timestep    0
Sample timestep    0
Sample timestep    0
Sample timestep    0
Shape torch.Size([32561, 109])
Sampling time: 188.396301984787
Sample path changed to ./synthetic/adult_ON_QnSC_200k/initial_tabddpm_ON_QnSC_200k_const_0_01.csv


/venv/main/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


CALCULATING COLUMN-WISE DENSITY ESTIMATION & PAIR-WISE COLUMN CORRELATION...
Generating report ...

(1/2) Evaluating Column Shapes: |██████████| 15/15 [00:00<00:00, 61.82it/s]|
Column Shapes Score: 98.04%

(2/2) Evaluating Column Pair Trends: |██████████| 105/105 [00:01<00:00, 61.81it/s]|
Column Pair Trends Score: 96.35%

Overall Score (Average): 97.2%

Error rate (%) of column-wise density estimation TABDDPM_ON_QNSC_200K_CONST_0_01: 1.957 ± 1.132
Error rate (%) of pair-wise column correlation score TABDDPM_ON_QNSC_200K_CONST_0_01: 3.646 ± 1.746


DONE! 

DRAW COLUMN VALUES DISTRIBUTIONS...


DONE!


...

TABDDPM_ON_QNSC_200K_MULT_10: ADULT_ON_QNSC_200K Dataset
Original Logistic: 0.657
Synthetic Logistic: 0.640
Original Tree: 0.660
Synthetic Tree: 0.639 

TABDDPM_ON_QNSC_200K_MULT_10 Accuracy Loss: ADULT_ON_QNSC_200K
Logistic: 2.607%
Tree: 3.200% 

CALCULATING COLUMN-WISE DENSITY ESTIMATION & PAIR-WISE COLUMN CORRELATION...
Generating report ...

(1/2) Evaluating Column Shapes: |██████████| 15/15 [00:00<00:00, 67.42it/s]|
Column Shapes Score: 98.37%

(2/2) Evaluating Column Pair Trends: |██████████| 105/105 [00:01<00:00, 65.87it/s]|
Column Pair Trends Score: 96.53%

Overall Score (Average): 97.45%

Error rate (%) of column-wise density estimation TABDDPM_ON_QNSC_200K_MULT_10: 1.629 ± 1.071
Error rate (%) of pair-wise column correlation score TABDDPM_ON_QNSC_200K_MULT_10: 3.474 ± 1.851


DONE! 

DRAW COLUMN VALUES DISTRIBUTIONS...


DONE!


100%|██████████| 36/36 [00:11<00:00,  3.21it/s]


Saving scores to  eval/mle/adult_ON_QnSC_200k/tabddpm_ON_QnSC_200k_mult_10.json
ROC - AUC обучения на синтетических данных TABDDPM_ON_QNSC_200K_MULT_10: 0.868 ± 0.008
adult_ON_QnSC_200k, tabddpm_ON_QnSC_200k_mult_10: 0.9286367521197229
DCR Score, a value closer to 0.5 is better
adult_ON_QnSC_200k-tabddpm_ON_QnSC_200k_mult_10, DCR Score = 0.6679463161450815
=========== All Features ===========
Data shape:  (32561, 110)
alpha precision: 0.942672, beta recall: 0.475534


In [11]:
final_metrics_table = []
for m in overall_metrics.keys():
    # Сбор таблички результатов
    tmp = pd.DataFrame([{'Model':model_short, 'Type':m, 'Data':dataname}])
    tmp.columns = pd.MultiIndex.from_tuples([('', i) for i in tmp.columns])
    result = [tmp]
    
    for metric_group in overall_metrics[m].keys():
        tmp = pd.DataFrame([overall_metrics[m][metric_group]])
        tmp.columns = pd.MultiIndex.from_tuples([(metric_group, i) for i in tmp.columns])
        result.append(tmp)
    result = pd.concat(result, axis = 1)
    final_metrics_table.append(result)
    
final_metrics_table = pd.concat(final_metrics_table)

In [12]:
pd.set_option('display.max_columns', None)
final_metrics_table.round(4).drop(columns=('base_metrics', 0)).sort_values([('similarity', 'Overall Score (Average), %')],
                                                                          ascending=False)

\
             Model                             Type                Data   
0  tabddpm_ON_QnSC  tabddpm_ON_QnSC_200k_const_0_25  adult_ON_QnSC_200k   
0  tabddpm_ON_QnSC     tabddpm_ON_QnSC_200k_mult_10  adult_ON_QnSC_200k   
0  tabddpm_ON_QnSC      tabddpm_ON_QnSC_200k_mult_6  adult_ON_QnSC_200k   
0  tabddpm_ON_QnSC  tabddpm_ON_QnSC_200k_const_0_01  adult_ON_QnSC_200k   
0  tabddpm_ON_QnSC     tabddpm_ON_QnSC_200k_const_0  adult_ON_QnSC_200k   
0  tabddpm_ON_QnSC      tabddpm_ON_QnSC_200k_mult_5  adult_ON_QnSC_200k   

              similarity                              \
  Column Shapes Score, % Column Pair Trends Score, %   
0                98.2458                     96.6649   
0                98.3706                     96.5264   
0                98.1516                     96.5212   
0                98.0435                     96.3541   
0                98.0746                     96.2226   
0                98.0725                     95.2966   

                              \
  Overall Score (Average), %   
0                    97.4554   
0                    97.4485   
0                    97.3364   
0                    97.1988   
0                    97.1486   
0                    96.6846   

                                                       \
  Error rate (%) of column-wise density estimation, %   
0                                              1.7542   
0                                              1.6294   
0                                              1.8484   
0                                              1.9565   
0                                              1.9254   
0                                              1.9275   

                                                           \
  Error rate (%) of column-wise density estimation std, %   
0                                                  0.9182   
0                                                  1.0712   
0                                                  1.2346   
0                                                  1.1320   
0                                                  1.1114   
0                                                  1.2965   

                                                           \
  Error rate (%) of pair-wise column correlation score, %   
0                                                  3.3351   
0                                                  3.4736   
0                                                  3.4788   
0                                                  3.6459   
0                                                  3.7774   
0                                                  4.7034   

                                                               \
  Error rate (%) of pair-wise column correlation score std, %   
0                                                      2.4591   
0                                                      1.8512   
0                                                      1.9479   
0                                                      1.7461   
0                                                      1.8915   
0                                                      2.7748   

                                         mle  \
  ROC - AUC обучения на синтетических данных   
0                                     0.8729   
0                                     0.8677   
0                                     0.8748   
0                                     0.8738   
0                                     0.8654   
0                                     0.8691   

                                                  detection     DCR  \
  ROC - AUC обучения на синтетических данных, std     Score   Score   
0                                          0.0072    0.9344  0.6701   
0                                          0.0084    0.9286  0.6679   
0                                          0.0070    0.9236  0.6687   
0                                          0.0073    0.9142  0.6678   

In [13]:
import os
if not os.path.exists(f'./eval/total/{dataname}'):
    os.makedirs(f'./eval/total/{dataname}')
final_metrics_table.to_csv(f'./eval/total/{dataname}/{model_short}{model_name_postfix}_final_metrcs_table.csv', index=False)